## Churn for High Net individuals on prepaid using usage based churn 

In [2]:
# imports 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('sumad_test').getOrCreate()

In [3]:
spark.version

'2.4.0'

In [60]:
from pyspark.sql import functions as F

In [164]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
#from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [5]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt
%matplotlib notebook 
plt.style.use('bmh')

In [6]:
!hadoop fs -ls

Found 7 items
drwx------   - sumad sumad          0 2018-12-04 03:00 .Trash
drwxr-xr-x   - sumad sumad          0 2019-01-16 01:34 .sparkStaging
-rw-r--r--   2 sumad sumad      21353 2018-11-30 01:58 2010-summary.json
drwxr-xr-x   - sumad sumad          0 2018-12-07 02:00 Data
drwxr-xr-x   - sumad sumad          0 2018-12-04 01:52 fractalUS
-rw-r--r--   2 sumad sumad   79156935 2019-01-16 01:41 telecom_churn_data.csv
drwxr-xr-x   - sumad sumad          0 2018-11-21 03:33 weblogs


In [9]:
!pwd

/home/sumad


## 1. Data Preparation

#### Read Data, Data Quality Check

#### had to pull data to local file system and then read

In [20]:
df_raw = spark.read.format("csv")\
 .option("header", "true")\
 .option("inferSchema", "true")\
 .load("/user/sumad/telecom_churn_data.csv")\
 .coalesce(5)
 #.where("Description IS NOT NULL")

In [21]:
df_raw.count()

99999

In [31]:
len(df_raw.columns)

226

In [26]:
type(df_raw)

pyspark.sql.dataframe.DataFrame

In [185]:
#df_raw.dtypes

In [35]:
df_sample = df_raw.limit(5).toPandas()

In [37]:
df_sample.to_csv('sample_telco.csv')

#### Filter high networth individuals

In [206]:
df_raw_1 = df_raw.withColumn('avg_rech_months_6_7', 
                             (F.col('total_rech_amt_6') + F.col('total_rech_amt_7'))/2)

In [207]:
th_amt = df_raw_1.select('avg_rech_months_6_7').summary('70%').collect()

In [208]:
df_raw_2 = (df_raw_1.withColumn('HNI_Ind',F.when(F.col('avg_rech_months_6_7') >= float(th_amt[0][1]),1)
                                .otherwise(0)).filter(F.col('HNI_Ind') == 1))

In [77]:
df_raw_2.count()

30011

#### Create churn indicator

In [209]:
df_raw_3 = df_raw_2.withColumn('Churn', F.when(((F.col("total_ic_mou_9") == 0) &
(F.col("total_og_mou_9") == 0) &
(F.col("vol_2g_mb_9") == 0) &
(F.col("vol_3g_mb_9") == 0)),1).otherwise(0))

In [210]:
df_raw_3.groupBy('Churn').count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|    1| 2593|
|    0|27418|
+-----+-----+



##### Churn rate of 9% in overall data can be seen

#### Drop all month 9 variables

In [211]:
month_9_cols = [x for x in df_raw_3.columns if(x.endswith('9'))]
print(len(df_raw_3.columns))
print(len(month_9_cols))

229
54


In [212]:
cols_to_keep_1 = [x for x in df_raw_3.columns if(x not in month_9_cols)]

In [213]:
df_raw_4 = df_raw_3.select(cols_to_keep_1)

In [214]:
len(df_raw_4.columns)

175

In [63]:
#df_raw_1.select(['total_rech_amt_6' ,'total_rech_amt_7', 'avg_rech_months_6_7']).show(5)

#### Data Quality Check

In [215]:
id_cols = ['mobile_number', 'circle_id' ]
target_col = 'Churn'
cat_cols = [x for x in df_raw_4.columns if(('date' in x) & (x not in id_cols) & (x!=target_col))]
num_cols = [x for x in df_raw_4.columns if x not in (cat_cols + id_cols + [target_col])]


In [216]:
print(len(cat_cols))
print(len(num_cols))

9
163


In [217]:
num_summary = df_raw_4.select(num_cols).summary().toPandas()

In [100]:
num_summary.to_csv('num_summary.csv')

In [98]:
cat_summary = df_raw_4.select(cat_cols).summary().toPandas()

In [99]:
cat_summary

,summary,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8
0,count,30011,29980,29854,29949,29897,29417,11397,11662,11754
1,mean,None,None,None,None,None,None,None,None,None
2,stddev,None,None,None,None,None,None,None,None,None
3,min,6/30/2014,7/31/2014,8/31/2014,6/1/2014,7/1/2014,8/1/2014,6/1/2014,7/1/2014,8/1/2014
4,25%,None,None,None,None,None,None,None,None,None
5,50%,None,None,None,None,None,None,None,None,None
6,75%,None,None,None,None,None,None,None,None,None
7,max,6/30/2014,7/31/2014,8/31/2014,6/9/2014,7/9/2014,8/9/2014,6/9/2014,7/9/2014,8/9/2014


#### Fix data issues

#### Remove numerical variables that 
- have any missing values 
- have 0 standard deviation 
#### Remove all date columns

In [218]:
num_summary.head()

,summary,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,...,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g,avg_rech_months_6_7,HNI_Ind
0,count,29897,29897,29897,30011,30011,30011,29695,29708,29073,...,11397,11662,11754,30011,30011,30011,30011,30011,30011,30011
1,mean,0.0,0.0,0.0,587.2844035520309,589.1354272766666,534.8574325080803,299.18471796598607,307.44728524303,276.23416778454106,...,0.8976046327981048,0.8898130680843767,0.8373319720946061,1264.0647762487088,129.43962647029423,135.12710239578823,121.36054813235148,6.562684682283162,696.6643564026524,1.0
2,stddev,0.0,0.0,0.0,442.72241348144973,462.8978139847736,492.2595861393725,462.20236658883493,483.24478606778746,471.5055178704068,...,0.3031808390001526,0.31313604065620493,0.36907821629148974,975.2631167127284,390.4785906443185,408.02439394814206,389.72603106156555,48.63865757176124,488.7820878731064,0.0
3,min,0,0,0,-2258.709,-2014.045,-945.808,0.0,0.0,0.0,...,0,0,0,180,0.0,0.0,0.0,0.0,368.5,1
4,25%,0,0,0,364.1,364.998,289.606,42.96,42.64,32.21,...,1,1,1,480,0.0,0.0,0.0,0.0,450.0,1


In [132]:
num_summary_ = num_summary.set_index('summary')

In [133]:
num_summary_.head()

,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,...,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g,avg_rech_months_6_7,HNI_Ind
summary,,,,,,,,,,,,,,,,,,,,,
count,29897,29897,29897,30011,30011,30011,29695,29708,29073,29695,...,11397,11662,11754,30011,30011,30011,30011,30011,30011,30011
mean,0.0,0.0,0.0,587.2844035520309,589.1354272766666,534.8574325080803,299.18471796598607,307.44728524303,276.23416778454106,422.380818993093,...,0.8976046327981048,0.8898130680843767,0.8373319720946061,1264.0647762487088,129.43962647029423,135.12710239578823,121.36054813235148,6.562684682283162,696.6643564026524,1.0
stddev,0.0,0.0,0.0,442.72241348144973,462.8978139847736,492.2595861393725,462.20236658883493,483.24478606778746,471.5055178704068,471.0962935132698,...,0.3031808390001526,0.31313604065620493,0.36907821629148974,975.2631167127284,390.4785906443185,408.02439394814206,389.72603106156555,48.63865757176124,488.7820878731064,0.0
min,0,0,0,-2258.709,-2014.045,-945.808,0.0,0.0,0.0,0.0,...,0,0,0,180,0.0,0.0,0.0,0.0,368.5,1
25%,0,0,0,364.1,364.998,289.606,42.96,42.64,32.21,141.41,...,1,1,1,480,0.0,0.0,0.0,0.0,450.0,1


In [219]:
mask = (num_summary_.loc['stddev'] != '0.0') | (num_summary_.loc['count'] == '30011')

In [220]:
sum(mask)

154

In [221]:
num_cols_sub = list(num_summary_.columns[mask].values)

In [160]:
#num_cols_sub

In [224]:
def drop_null_columns(df):
    """
    This function drops all columns which contain null values.
    :param df: A PySpark DataFrame
    """
    null_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
    to_drop = [k for k, v in null_counts.items() if v > 0]
    df = df.drop(*to_drop)
    return df

# Drops column b2, because it contains null values
final_df = drop_null_columns(df_raw_4)

In [225]:
len(final_df.columns)

50

In [232]:
#final_df.columns

#### 2. Feature Engineering 

In [228]:
final_df = final_df.withColumn('arpu_6_7', 
                             (F.col('arpu_6') + F.col('arpu_7')))

In [233]:
final_features_1 = [x for x in (final_df.columns ) if ((x not in (id_cols + [target_col])) & 
                                                     (x in num_cols_sub))]

In [234]:
len(final_features)

46

#### Split into training and test set 

In [235]:
from pyspark.ml.feature import VectorAssembler

In [169]:
#final_features_1 = [x for x in final_features if (x not in (id_cols + [target_col]))]

In [179]:
#final_features_1

In [236]:
assembler = VectorAssembler(inputCols= final_features,outputCol="features")

In [237]:
output = assembler.transform(df_raw_4).select('features', 'Churn')

In [238]:
output.show(5)

+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|(46,[0,1,2,3,4,5,...|    1|
|[378.721,492.223,...|    0|
|(46,[0,1,2,3,4,5,...|    0|
|(46,[0,1,2,3,4,5,...|    0|
|(46,[0,1,2,3,4,6,...|    0|
+--------------------+-----+
only showing top 5 rows



In [239]:
train_data,test_data = output.randomSplit([0.7,0.3])

#### 3. Model Training  
- Model categories to be considered  
- Handling class imbalance 
- Parameter tuning using Cross Validation , choice of eval metric

In [240]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [243]:
dtc_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')
reg_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')

In [244]:

log_reg = LogisticRegression(featuresCol='features',labelCol='Churn')

paramGrid_reg = ParamGridBuilder()\
    .addGrid(log_reg.regParam, [0.1, 0.01]) \
    .build()

crossval_reg = CrossValidator(estimator=log_reg,
                          estimatorParamMaps=paramGrid_reg,
                          evaluator=reg_eval,
                          numFolds=5, parallelism = 2)
cvModel_reg = crossval_reg.fit(train_data)

In [245]:
dtc = DecisionTreeClassifier(labelCol='Churn',featuresCol='features')

In [249]:
#dtc.explainParams()

In [250]:
dtc = DecisionTreeClassifier(labelCol='Churn',featuresCol='features')

paramGrid_dtc = ParamGridBuilder()\
    .addGrid(dtc.maxDepth, [2,5])\
    .build()

crossval_dtc = CrossValidator(estimator=dtc,
                          estimatorParamMaps=paramGrid_dtc,
                          evaluator=dtc_eval,
                          numFolds=5, parallelism = 2)
cvModel_dtc = crossval_dtc.fit(train_data)

#### 4. Model Evaluation on test
- Use a single metric 
- Visualize where which model is better

In [251]:
result_dtc = cvModel_dtc.transform(test_data)
result_reg = cvModel_reg.transform(test_data)

In [ ]:
dtc_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')
reg_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')

In [253]:
print(AUC_reg, AUC_dtc)

0.5650942276050412 0.7328874979503447
